In [1]:
import itertools

import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx

import meshplot as mp
import pyvista as pv
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

from src import shapes

# Define The Figure

In [2]:
def diffeo(vertices, a=0.2, b=1.1, c=0.2, d=1.2, e=0.2, f=0.3):
    v = vertices.copy()
    x, y, z = v[:, 0], v[:, 1], v[:, 2]

    v[:, 0] = x + a*np.sin(b*y) + c*np.sin(d*z)
    v[:, 1] = y + e*np.sin(f*x)

    return v

def cylindrical_twist(vertices, k=0.2, mode="x", scale=0.5):
    """
    Nonlinear cylindrical twist diffeomorphism on R^3.

    vertices: (n,3) array
    mode:
      - "z": angle depends on z  (theta = k * tanh(z/scale))
      - "r": angle depends on radius r (theta = k * tanh(r/scale))
    k: twist strength (radians, roughly bounded by +/-k for tanh)
    scale: controls how quickly tanh saturates
    """
    if mode == 'x':
        v = vertices[:, [1, 2, 0]]
        v = cylindrical_twist(v, k=k, mode="z", scale=scale)
        v = v[:, [2, 0, 1]]
        return v
        
    v = vertices.copy()
    x, y, z = v[:, 0], v[:, 1], v[:, 2]

    r = np.sqrt(x*x + y*y)

    if mode == "z":
        theta = k * np.tanh(z / scale)
    elif mode == "r":
        theta = k * np.tanh(r / scale)
    else:
        raise ValueError('mode must be "z" or "r"')

    c, s = np.cos(theta), np.sin(theta)

    v[:, 0] = c * x - s * y
    v[:, 1] = s * x + c * y
    # z unchanged
    return v

In [3]:
n, m = 13, 12
vertices, faces = shapes.get_halftori_bouquet(leaves=3, n=n, m=m, l0=0.9, glue=True)
vertices, faces = shapes.split_large_edges(vertices, faces, max_length=1.0)


#vertices += 0.05*np.random.normal(size=vertices.shape)

print(f'faces.shape = {faces.shape}')

faces.shape = (1322, 3)


In [4]:
n, m = 17, 16
vertices0, faces0 = shapes.get_halftori_bouquet(leaves=3, n=n, m=m, l0=1.0, glue=True)
#vertices3, faces3 = shapes.get_halftori_bouquet(leaves=2, n=n, m=m, l0=0.6, glue=False)

n0, n1 = 12, 12
r0, r1 = 0.8, 0.6
vertices1, faces1 = shapes.get_couple_linked_tori(n0=n0, n1=n1, r0=r0, r1=r1)

vertices0 += np.array([vertices0[:, 0].min(), 0, 0])
vertices1 += np.array([vertices1[:, 0].max(), 0, 0])

vertices, faces, _ = shapes.merge_meshes_with_weld(vertices0, faces0, vertices1, faces1)
vertices, faces = shapes.split_large_edges(vertices, faces, max_length=1.0)

vertices = cylindrical_twist(vertices, mode='x', scale=1.0, k=0.4)

In [5]:
faces_pv = np.hstack([np.full((faces.shape[0], 1), 3, dtype=faces.dtype), faces]).ravel()

mesh = pv.PolyData(vertices, faces_pv)

pl = pv.Plotter(window_size=(600, 600))
pl.add_mesh(mesh, smooth_shading=False, show_edges=True, opacity=0.6)
pl.show()

Widget(value='<iframe src="http://localhost:33145/index.html?ui=P_0x79c673e04b60_0&reconnect=auto" class="pyvi…

# Detect Nonmanifold Structure

In [6]:
def iter_degree2_chains(G):
    """
    Yield chains (as lists of nodes) in an undirected graph G such that:
      - endpoints have degree != 2
      - internal nodes (if any) have degree == 2
    Each chain is yielded once.
    """
    deg = dict(G.degree())
    is_endpoint = {v: (deg[v] != 2) for v in G.nodes}

    visited_edges = set()

    def mark_edge(a, b):
        e = (a, b) if a <= b else (b, a)
        visited_edges.add(e)

    def edge_marked(a, b):
        e = (a, b) if a <= b else (b, a)
        return e in visited_edges

    for u in G.nodes:
        if not is_endpoint[u]:
            continue  # only start from endpoint-ish nodes (deg != 2)

        for v in G.neighbors(u):
            if edge_marked(u, v):
                continue

            path = [u, v]
            mark_edge(u, v)

            prev, cur = u, v
            while deg[cur] == 2:
                # continue through the unique "other" neighbor
                n1, n2 = G.neighbors(cur)
                nxt = n2 if n1 == prev else n1

                if edge_marked(cur, nxt):
                    # can happen in some multi-path situations; stop safely
                    break

                path.append(nxt)
                mark_edge(cur, nxt)
                prev, cur = cur, nxt

            # now cur has degree != 2 OR we broke early
            if deg[path[0]] != 2 and deg[path[-1]] != 2:
                yield path


In [7]:
def get_boundaries1(faces, cycle_start_strategy=None):
    """
    """
    # get edge degrees
    edge_degrees = dict()
    for edge in np.concatenate(np.unique(np.sort(faces, axis=1), axis=0)[:, [[0, 1],[0, 2], [1, 2]]]):
        try:
            edge_degrees[tuple(edge)] += 1
        except KeyError:
            edge_degrees.update({tuple(edge) : 1})
        
    # define boundaries
    graph_boundaries = nx.Graph()
    graph_boundaries.add_edges_from([edge for edge, degree in edge_degrees.items() if degree != 2])
    
    # boundaries dimension 1
    boundaries1 = []
    for chain in iter_degree2_chains(graph_boundaries):
        boundaries1.append(np.array(chain))
    
    # cyclic boundaries dimension 1
    if cycle_start_strategy is None:
        def cycle_start_strategy(component, **kwargs):
            return list(component)[0]
    graph_cyclic_boundaries = graph_boundaries.copy()
    for boundary in boundaries1:
        graph_cyclic_boundaries.remove_nodes_from(boundary)
    for component in nx.connected_components(graph_cyclic_boundaries):
        cycle_start = cycle_start_strategy(component=component, faces=faces)
        source, target = graph_cyclic_boundaries.neighbors(cycle_start)
        graph_cycle = graph_cyclic_boundaries.subgraph(component)
        chain = nx.shortest_path(graph_cycle, source, target)
        chain = np.concatenate([[cycle_start], chain, [cycle_start]])
        boundaries1.append(chain)
    
    return boundaries1
        

In [8]:
get_boundaries1(faces)

[array([ 267, 1154,  265,  275,  278,  305,  326,  350,  383,  404,  427,
         455,  476,  509,  533,  554,  581,  584,  593, 1156,  595]),
 array([ 267, 1155,  268,  277,  280,  307,  328,  352,  385,  406,  439,
         457,  478,  511,  535,  556,  583,  586,  596, 1158,  595]),
 array([ 267, 1157,  266,  276,  279,  306,  327,  351,  384,  405,  428,
         456,  477,  510,  534,  555,  582,  585,  594, 1161,  595]),
 array([ 267, 1131,  595]),
 array([985, 979, 959, 942, 921, 909, 908, 910, 922, 943, 960, 980, 985]),
 array([ 985, 1002, 1020, 1043, 1060, 1076, 1086, 1077, 1061, 1044, 1021,
        1003,  985])]

In [9]:
def get_conic_vertices(faces):
    """
    """
    conic_vertices = []
    for vertex in np.unique(faces):
        faces_vertex = faces[(faces == vertex).any(axis=1)]
        connections = faces_vertex[:, None, :, None] == faces_vertex[None, :, None, :]
        connections = connections.any(axis=-1).sum(axis=-1) == 2
        n_components, _ = sp.sparse.csgraph.connected_components(connections, directed=False)
        if n_components > 1:
            conic_vertices.append(vertex)
    conic_vertices = np.array(conic_vertices)
    return conic_vertices

In [10]:
def define_distances_on_graph(graph: nx.Graph, values, current_nodes=[], closest_nodes=None, edge_weight='weight'):
    """
    """
    if len(current_nodes) == 0:
        if closest_nodes is None:
            return values
        else:
            return values, closest_nodes
    next_nodes = []
    for current_node in current_nodes:
        neighbors = np.array(list(graph.neighbors(current_node)))
        lengths = np.array([graph[current_node][neighbor][edge_weight] for neighbor in neighbors])
        update_condition = values[current_node] + lengths < values[neighbors]
        values[neighbors[update_condition]] = values[current_node] + lengths[update_condition]
        if closest_nodes is not None:
            closest_nodes[neighbors[update_condition]] = closest_nodes[current_node]
        next_nodes.extend(neighbors[update_condition])
    next_nodes = np.unique(next_nodes)
    return define_distances_on_graph(graph, values, current_nodes=next_nodes, closest_nodes=closest_nodes, edge_weight=edge_weight)


    

In [11]:
from fractions import Fraction
from math import gcd
from functools import reduce

def approx_ratio_integers(ls, max_den=1000, ref="min"):
    assert all(x > 0 for x in ls)
    k = min(range(len(ls)), key=ls.__getitem__) if ref=="min" else 0
    rs = [x / ls[k] for x in ls]

    frs = [Fraction(r).limit_denominator(max_den) for r in rs]
    dens = [f.denominator for f in frs]

    # lcm of denominators
    def lcm(a, b): return a // gcd(a, b) * b
    L = reduce(lcm, dens, 1)

    ns = [int(f.numerator * (L // f.denominator)) for f in frs]

    # optional: divide by gcd to simplify
    g = reduce(gcd, ns)
    ns = [n // g for n in ns]
    return ns


In [12]:

G = nx.Graph()
G.add_node(1, weight=4)
G.add_edge(1, 2, length=3.5)

l = G[1][2]["length"]
w = G.nodes[1]['weight']
print(l, w)  # 3.5

3.5 4


In [13]:
a = []
a.extend(np.arange(4))
a

[np.int64(0), np.int64(1), np.int64(2), np.int64(3)]

In [14]:
class NonmanifoldStructure:
    def __init__(self, faces, cycle_start_strategy_with_respect_to_conic=None):
        """
        """
        # 
        self.faces = np.unique(np.sort(faces, axis=1), axis=0)

        # 
        conic_vertices = get_conic_vertices(self.faces)
        
        # 
        if cycle_start_strategy_with_respect_to_conic is None:
            def cycle_start_strategy(component, **kwargs):
                conic_component = set(conic_vertices) & set(component)
                if len(conic_component) > 0:
                    return list(conic_component)[0]
                return list(component)[0]
        else:
            cycle_start_strategy = None

        # 
        self.boundaries1 = get_boundaries1(self.faces, cycle_start_strategy=cycle_start_strategy)
        for conic_vertex in conic_vertices:
            for i in range(len(self.boundaries1)):
                conic_index = np.argwhere(self.boundaries1[i][1:-1] == conic_vertex)
                if conic_index.size > 0:
                    conic_index = conic_index[0][0] + 1
                    chain0 = self.boundaries1[i][:conic_index + 1]
                    chain1 = self.parts1[i][conic_index:]
                    self.boundaries1[i] = chain0
                    self.boundaries1.append(chain1)

        self.boundaries0 = np.unique(np.concatenate([chain[[0, -1]] for chain in self.boundaries1] + [conic_vertices]))

        # 
        connectivity_graph = nx.Graph()
        connectivity_graph.add_edges_from(np.concatenate(self.faces[:, [[0, 1], [0, 2], [1, 2]]]))
        connectivity_graph.remove_nodes_from(self.boundaries0)
        connectivity_graph.remove_nodes_from(np.concatenate(self.boundaries1))
        
        self.faces_parts = -np.ones(self.faces.shape[0])
        self.n_parts = 0
        for i, component in enumerate(nx.connected_components(connectivity_graph)):
            self.faces_parts[np.argwhere(np.isin(self.faces, list(component)).any(axis=1))] = i
            self.n_parts += 1
         
    def get_face_parts(self, faces):
        mask = (self.faces[:, None, :] == np.sort(faces, axis=1)[None, :, :]).all(axis=2)
        indices = mask.argmax(axis=0)
        return self.faces_parts[indices]

    def iterate_parts(self):
        for i in range(self.n_parts):
            vertices = np.unique(self.faces[self.faces_parts == i])
            yield vertices


    def define_mesh_faces_indices(self, mesh: pv.core.pointset.PolyData, title='part'):
        """
        """
        mesh.cell_data[title] = self.get_face_parts(mesh.faces.reshape(-1, 4)[:, 1:])
        return mesh
    
    def get_parts_boundary_vertices(self):
        """
        """
        boundary_vertices = np.unique(np.concatenate(self.boundaries1 + self.boundaries0))
        part_boundary_vertices = [np.intersect1d(boundary_vertices, np.unique(part)) for part in self.faces_parts]
        return part_boundary_vertices
    
    def get_distance_to_boundaries(self, vertices, define_closest_nodes=False):
        """
        """
        boundary_vertices = np.unique(np.concatenate(self.boundaries1 + [self.boundaries0]))
        values = +np.inf*np.ones(self.faces.max() + 1)
        values[boundary_vertices] = 0
        if define_closest_nodes:
            closest_nodes = -np.ones(self.faces.max() + 1)
            closest_nodes[boundary_vertices] = boundary_vertices
        else:
            closest_nodes = None

        vertices = np.array(vertices)
        if len(values) != len(vertices):
            raise ValueError(f"Expected the vartices shape ({len(values)}, d), but got an array shape {vertices.shape}.")
        
        edges = np.unique(np.sort(np.concatenate(self.faces[:, [[0, 1], [0, 2], [1, 2]]]), axis=1), axis=0)
        lengths = np.linalg.norm(vertices[edges[:, 0]] - vertices[edges[:, 1]], axis=-1)

        graph = nx.Graph()
        graph.add_nodes_from(np.arange(len(values)))
        graph.add_weighted_edges_from((u, v, w) for (u, v), w in zip(edges, lengths))
        
        return define_distances_on_graph(graph, values, current_nodes=boundary_vertices, closest_nodes=closest_nodes, edge_weight='weight')
    
    def get_boundary_lengths(self, vertices):
        """
        """
        boundary_lengths = np.array([np.linalg.norm(vertices[chain[1:]] - vertices[chain[:-1]], axis=-1).sum() for chain in self.boundaries1])
        return boundary_lengths
    
    def aggregate_values_in_parts(self, values, agg=np.max):
        """
        """
        parts_vertices = [np.unique(faces[self.faces_parts == i]) for i in range(self.n_parts)]
        agg_values = [agg(values[part]) for part in parts_vertices]
        return agg_values
    
    def map_boundaries(self, vertices=None):
        """
        """
        boundaries_maps = []
        for chain in self.boundaries1:
            if vertices is None:
                l = np.arange(len(chain))
            else:
                l = np.linalg.norm(vertices[chain[1:]] - vertices[chain[:-1]], axis=-1)
                l = np.append(0, np.cumsum(l))
            boundaries_maps.append(l/l.max())
        return boundaries_maps

    def get_good_values(self, vertices, max_den=2):
        """
        """
        lengths = self.get_boundary_lengths(vertices)
        distances_to_boundaries, closest_boundaries = self.get_distance_to_boundaries(vertices, define_closest_nodes=True)
        radiuses = self.aggregate_values_in_parts(distances_to_boundaries, agg=np.max)
        
        lengths_radiuses_int = approx_ratio_integers(np.concatenate([lengths, radiuses]), max_den=max_den)
        lengths_int = lengths_radiuses_int[:len(lengths)]
        radiuses_int = lengths_radiuses_int[len(lengths):]

        boundaries_maps = self.map_boundaries(vertices)

        # angular values
        values_boundaries = np.empty(len(vertices))
        values_boundaries[self.boundaries0] = 1
        for chain, boundary_map, l_int in zip(self.boundaries1, boundaries_maps, lengths_int):
            #values_boundaries[chain] = np.sin(l_int*2*np.pi*boundary_map)
            values_boundaries[chain] = np.sin(2*np.pi*boundary_map)
        assert (closest_boundaries == closest_boundaries.astype(int)).all()
        values_boundaries = values_boundaries[closest_boundaries.astype(int)]
        print('values_boundaries.shape = {values_boundaries.shape}')

        # radial values
        values_radial = np.empty(len(vertices))
        for part, r, r_int in zip(self.iterate_parts(), radiuses, radiuses_int):
            #values_radial[part] = np.sin(2*np.pi*r_int*distances_to_boundaries[part]/r)
            values_radial[part] = np.sin(2*np.pi*distances_to_boundaries[part]/r)
        print('values_radial.shape = {values_radial.shape}')

        values = values_boundaries + values_radial
        print('values.shape = {values.shape}')
        return values

In [15]:
nm = NonmanifoldStructure(faces)

nm.boundaries0, nm.boundaries1

(array([267, 595, 860, 985]),
 [array([ 267, 1154,  265,  275,  278,  305,  326,  350,  383,  404,  427,
          455,  476,  509,  533,  554,  581,  584,  593, 1156,  595]),
  array([ 267, 1155,  268,  277,  280,  307,  328,  352,  385,  406,  439,
          457,  478,  511,  535,  556,  583,  586,  596, 1158,  595]),
  array([ 267, 1157,  266,  276,  279,  306,  327,  351,  384,  405,  428,
          456,  477,  510,  534,  555,  582,  585,  594, 1161,  595]),
  array([ 267, 1131,  595]),
  array([985, 979, 959, 942, 921, 909, 908, 910, 922, 943, 960, 980, 985]),
  array([ 985, 1002, 1020, 1043, 1060, 1076, 1086, 1077, 1061, 1044, 1021,
         1003,  985])])

In [16]:
mesh = nm.define_mesh_faces_indices(mesh)

pl = pv.Plotter(window_size=(600, 500))
pl.add_mesh(mesh, smooth_shading=False, show_edges=True, opacity=0.8, scalars='part', categories=True, colormap='rainbow')

pl.add_points(vertices[nm.boundaries0], color='white', point_size=18, render_points_as_spheres=True)

for chain in nm.boundaries1:
    if len(chain) < 2:
        continue
    pts = vertices[chain]                 # (k, 3)
    line = pv.lines_from_points(pts, close=False)
    pl.add_mesh(line, color="white", line_width=5, render_lines_as_tubes=True)

pl.show()

Widget(value='<iframe src="http://localhost:33145/index.html?ui=P_0x79c672d2ec00_1&reconnect=auto" class="pyvi…

In [17]:
values = nm.get_distance_to_boundaries(vertices)

mesh.point_data['distance_to_boundary'] = values

pl = pv.Plotter(window_size=(600, 500))
pl.add_mesh(mesh, smooth_shading=False, show_edges=True, opacity=1.0, categories=True, scalars='distance_to_boundary',  colormap='turbo')

pl.add_points(vertices[nm.boundaries0], color='white', point_size=18, render_points_as_spheres=True)

for chain in nm.boundaries1:
    if len(chain) < 2:
        continue
    pts = vertices[chain]                 # (k, 3)
    line = pv.lines_from_points(pts, close=False)
    pl.add_mesh(line, color="white", line_width=5, render_lines_as_tubes=True)

pl.show()

Widget(value='<iframe src="http://localhost:33145/index.html?ui=P_0x79c65c0dad50_2&reconnect=auto" class="pyvi…

In [18]:
values = nm.get_good_values(vertices)

mesh.point_data['good'] = values

pl = pv.Plotter(window_size=(600, 500))
pl.add_mesh(mesh, smooth_shading=False, show_edges=True, opacity=1.0, categories=True, scalars='good',  colormap='turbo')

pl.add_points(vertices[nm.boundaries0], color='white', point_size=18, render_points_as_spheres=True)

for chain in nm.boundaries1:
    if len(chain) < 2:
        continue
    pts = vertices[chain]                 # (k, 3)
    line = pv.lines_from_points(pts, close=False)
    pl.add_mesh(line, color="white", line_width=5, render_lines_as_tubes=True)

pl.show()

values_boundaries.shape = {values_boundaries.shape}
values_radial.shape = {values_radial.shape}
values.shape = {values.shape}


Widget(value='<iframe src="http://localhost:33145/index.html?ui=P_0x79c65c1072c0_3&reconnect=auto" class="pyvi…

# Morse Smale

In [19]:
from src.ms import MorseSmale
from src.vis import plot_paths

In [20]:
ms = MorseSmale(nm.faces, nm.get_good_values(vertices), vertices)


values_boundaries.shape = {values_boundaries.shape}
values_radial.shape = {values_radial.shape}
values.shape = {values.shape}


In [21]:
nm.get_good_values(vertices)

values_boundaries.shape = {values_boundaries.shape}
values_radial.shape = {values_radial.shape}
values.shape = {values.shape}


array([ 0.64779307,  0.8668575 , -0.15584918, ...,  0.80736749,
        1.79642311,  1.80661604], shape=(1466,))

In [22]:
list(ms.iterate_paths()), len(list(ms.iterate_paths()))

([array([   2, 1152,    1]),
  array([   2, 1153,    3]),
  array([   2, 1151,    0]),
  array([25,  7,  0]),
  array([ 25,  61,  97, 151, 196]),
  array([26,  8,  1]),
  array([ 26,  62,  98, 152, 197]),
  array([ 28,  64, 100, 154, 199]),
  array([28,  9,  0]),
  array([ 29,  65, 101, 155, 200]),
  array([29, 10,  1]),
  array([ 32,  68, 104, 158, 203]),
  array([32, 14,  3]),
  array([ 33,  69, 105, 159, 204]),
  array([33, 15,  3]),
  array([ 198, 1282,  196]),
  array([ 198, 1174,  197]),
  array([ 201, 1283,  203]),
  array([ 201, 1274,  199]),
  array([ 202, 1284,  204]),
  array([ 202, 1287,  200]),
  array([ 265, 1211]),
  array([265, 238, 196]),
  array([265, 240, 199]),
  array([266, 241, 200]),
  array([266, 239, 197]),
  array([ 266, 1465]),
  array([ 266, 1212]),
  array([268, 245, 203]),
  array([268, 246, 204]),
  array([ 268, 1215]),
  array([278, 253, 205, 196]),
  array([ 278, 1138]),
  array([279, 272, 214, 208, 200]),
  array([ 279, 1136]),
  array([280, 273, 215, 

In [23]:
pl = plot_paths(ms)
pl.show()

Widget(value='<iframe src="http://localhost:33145/index.html?ui=P_0x79c65c5be780_4&reconnect=auto" class="pyvi…